In [ ]:
import os
import hopsworks

In [ ]:
with open('data/hopsworks-api-key.txt', 'r') as file:
    os.environ["HOPSWORKS_API_KEY"] = file.read().rstrip()
    
project = hopsworks.login()
fs = project.get_feature_store()

In [ ]:
grailed_items_fg = fs.get_feature_group(
    name='draft_grailed_items',
    version=1,
)

In [ ]:
grailed_items_fg.features

## Create feature view

In [ ]:
selected_features = grailed_items_fg.select(['sold_price', 'designers_title_embedding'])
selected_features.show(10)

In [ ]:
feature_view = fs.get_or_create_feature_view(
    name='grailed_items_fv',
    version=1,
    labels=['sold_price'],
    query=selected_features,
)

In [ ]:
X_train, X_test, y_train, y_test = feature_view.train_test_split(test_size=0.2)

In [ ]:
X_train

## Create a predictive model

In [ ]:
from xgboost import XGBRegressor

In [ ]:
import numpy as np

# Convert the embeddings column into a numpy array of separate columns
X_train_array = np.vstack(X_train['designers_title_embedding'].to_numpy())
X_test_array = np.vstack(X_test['designers_title_embedding'].to_numpy())

# Now train the model with the reshaped data
xgb_regressor = XGBRegressor()
xgb_regressor.fit(X_train_array, y_train)

In [ ]:
from sklearn.metrics import mean_squared_error, r2_score

# Predicting target values on the test set
y_pred = xgb_regressor.predict(X_test_array)

# Calculating Mean Squared Error (MSE) using sklearn
mse = mean_squared_error(y_test.iloc[:,0], y_pred)
print("MSE:", mse)

# Calculating R squared using sklearn
r2 = r2_score(y_test.iloc[:,0], y_pred)
print("R squared:", r2)

In [ ]:
# TODO: Is the order correct?
predicted_df = y_test
predicted_df['predicted_price'] = y_pred

In [ ]:
predicted_df

In [ ]:
# TODO: Plot performance, consider saving the plots as images

In [ ]:
# TODO: Plot feature importance

## Model registry

In [ ]:
# TODO: Save the model locally

In [ ]:
# TODO: Save the model to model registry